# Approximation Algorithms

In the last notebook, we saw one algorithm to solve the Traveling Salesperson Problem. This time, we'll implement some algorithms which will give us approximate solutions to the TSP problem in polynomial time.

### If you're using Datahub:
* Run the cell below **and restart the kernel if needed**

### If you're running locally:
You'll need to perform some extra setup.
#### First-time setup
* Install Anaconda following the instructions here: https://www.anaconda.com/products/distribution 
* Create a conda environment: `conda create -n cs170 python=3.8`
* Activate the environment: `conda activate cs170`
    * See for more details on creating conda environments https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html
* Install pip: `conda install pip`
* Install jupyter: `conda install jupyter`

#### Every time you want to work
* Make sure you've activated the conda environment: `conda activate cs170`
* Launch jupyter: `jupyter notebook` or `jupyter lab` 
* Run the cell below **and restart the kernel if needed**

In [187]:
# Install dependencies
!pip install -r requirements.txt --quiet

In [188]:
import otter
assert (otter.__version__ >= "5.4.1"), "Please reinstall the requirements and restart your kernel."

grader = otter.Notebook("hw12-coding.ipynb")
import networkx as nx
import pickle

with open('tests_1.pkl', 'rb') as f:
    test_data = pickle.load(f)

rng_seed = 0

# The Traveling Salesperson Problem

In this notebook, we will revisit the Traveling Salesperson Problem, which asks the following question: _Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?_

The problem can be formally defined as follows:

```
Input: An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
Output: An ordered list of cities [c1, c2, ... cn] that defines the shortest tour which passes through all the cities, starting and ending at the same city.
```

TSP is an NP-complete problem, and unless P=NP, there is no polynomial-time algorithm that finds exact solutions to the problem. You may remember that the Dynamic Programming algorithm we implemented in the last homework was very slow :)

This time, we will focus on efficient ways to find approximate solutions. 

We have provided a convenience function that, given an input matrix and a list of cities, evaluates the length of the path that passes through all of the cities in the list in order.

In [215]:
def validate_tour(tour, matrix):
    """Returns the length of the tour if it is valid, -1 otherwise
    """
    n = len(tour)
    cost = 0
    for i in range(n):
        if matrix[tour[i-1]][tour[i]] == float('inf'):
            return -1
        cost += matrix[tour[i-1]][tour[i]]
    return cost

### Q1a) Greedy Solution from Designated Home
Implement a greedy solution, which starts at city `home` and greedily chooses the closest city that has not been visited yet, until all cities have been visited. Return the path as a list of cities on that path, starting and ending at `path[0]`. **For example, to represent the cycle `0 -> 1 -> 2 -> 3 -> 0`, return the list `[0, 1, 2, 3]`.**  You may break ties arbitrarily.

In [216]:
def tsp_greedy(matrix, home):
    """
    A greedy implementation of TSP, starting and ending at home.
    
    Args:
        matrix: List[List[float]]
            An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
        home: int
            The index of the city to start and end at.
    
    Returns:
        path: List[int] 
            A list corresponding to the order in which to visit cities, starting from path[0] and ending 
            at path[-1] before returning to path[0]. path[0] should be home.    
    """
    n = len(matrix)
    visited = [False] * n
    tour = [home]
    curr_city = home

    visited[home] = True

    while len(tour) < n + 1:
        min_distance = float('inf')
        c_city = None

        for city in range(n):
            if not visited[city] and matrix[curr_city][city] < min_distance:
                min_distance = matrix[curr_city][city]
                c_city = city

        if c_city is None:
            break

        visited[c_city] = True
        tour.append(c_city)
        curr_city = c_city

    return tour

In [217]:
grader.check("q1a")

Testing on 5 cities...
Testing on 10 cities...
Testing on 20 cities...
Testing on 40 cities...
Testing on 100 cities...
All test cases passed!


q1a results: All test cases passed!

### Q1b) Greedy Solution 
An easy way to improve over the original greedy solution is to try your greedy solution on all of the possible starting locations and choose the best one. Implement a general greedy solution, which runs the Q1a implementation on all possible home locations, and returns the best overall path.

_Your solution should take around 8 lines of code.__

In [218]:
def tsp_greedy_general(matrix):
    """
    A generalized greedy implementation of TSP.
    
    Args:
        matrix: List[List[float]]
            An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
    
    Returns:
        path: List[int] 
            A list corresponding to the order in which to visit cities, starting from path[0] and ending 
            at path[-1] before returning to path[0].  
    """
    n = len(matrix)
    best_path = None
    best_cost = float('inf')
    
    for home in range(n):
        tour = tsp_greedy(matrix, home)
        cost = validate_tour(tour, matrix)
        if cost != -1 and cost < best_cost:
            best_cost = cost
            best_path = tour
    
    return best_path

In [219]:
grader.check("q1b")

Testing on 5 cities...
Testing on 10 cities...
Testing on 20 cities...
Testing on 40 cities...
Testing on 100 cities...
All test cases passed!


q1b results: All test cases passed!

### Q2) Approximation Algorithm for Metric TSP

When NP-complete problems are given specific constraints, they are sometimes easier to approximate. For this question, we will focus on a special variant of TSP called the **metric TSP**, where distances satisfy the following three properties:
1. Distances are non-negative: $d(i, j) \geq 0$
2. Distances are symmetric: $d(i, j) = d(j, i)$
3. Distances satisfy the following inequality:

$$ \forall i, j, k \in V,\; d(i, k) \leq d(i, j) + d(j, k)$$

_(This is called the triangle inequality, and all mathematical distance metrics obey it, which is why this is called the metric TSP!)_

In other words, the graph is complete and the shortest path from one city to another city is always the direct path.

The Metric TSP problem is still NP-complete, but the following approximation returns a path that is guaranteed to be **at most twice the length of the optimal path**:

* Generate a minimum spanning tree of the graph.
* Run depth-first search on the minimum spanning tree.
* Return the nodes in the order that you found them with depth first search (i.e. by preorder number).

See DPV Section 9.2 for more details: https://people.eecs.berkeley.edu/~vazirani/algorithms/chap9.pdf#page=12

Implement this approximation algorithm below.

**For this problem, run depth-first search starting at node 0, and explore neighbors in numerical order.** 

_Feel free to reuse code from previous coding homework assignments, but please don't use any external library imports for this part. If you want to reuse your Kruskal's code to generate an MST, you should reconfigure it to take in an adjacency matrix instead of an adjacency list._

In [220]:
class UnionFind:
    def __init__(self, n):
        self.n = n
        self.parents = [i for i in range(n)]
        self.rank = [1]*n
    
    def find(self, i):
        assert i >= 0 and i <= self.n-1, f"Node {i} is not in the data structure. Only nodes {0} through {self.n-1} exist."
        if i != self.parents[i]:
            i = self.find(self.parents[i])
        return i
    
    def union(self, pi, pj):
        assert pi >= 0 and pi <= self.n-1, f"Node {pi} is not in the data structure. Only nodes {0} through {self.n-1} exist."
        assert pj >= 0 and pj <= self.n-1, f"Node {pj} is not in the data structure. Only nodes {0} through {self.n-1} exist."
        x = self.find(pi)
        y = self.find(pj)
        if x == y: return
        if self.rank[x] > self.rank[y]:
            self.parents[y] = x
        else:
            self.parents[x] = y
            if self.rank[x] == self.rank[y]: 
                self.rank[y] += 1

In [221]:
def kruskal(G):
    T = []
    edges = []
    
    for u in range(len(G)):
        for v, w in G[u]:
            edges.append((w, u, v))
    edges.sort()
    
    UF = UnionFind(len(G))
                         
    for e in edges:
        u,v = e[1],e[2]
        if UF.find(u) != UF.find(v):
            UF.union(u,v)
            T.append((u,v))

    if len(T) != len(G) - 1:
        return []

    return T

In [222]:
def matrix_to_adj_list(matrix):
    adj_list = []
    n = len(matrix)

    for i in range(n):
        neighbors = []
        for j in range(n):
            if matrix[i][j] != -1:
                neighbors.append((j, matrix[i][j]))
        adj_list.append(neighbors)

    return adj_list

def construct_adj_list(edges):
    adj_list = [[] for _ in range(len(edges) + 1)]
    for u, v in edges:
        adj_list[u].append(v)
        adj_list[v].append(u) 
    return adj_list

def dfs(adj_list, u, visited, path):
    visited[u] = True
    path.append(u)
    for v in adj_list[u]:
        if not visited[v]:
            dfs(adj_list, v, visited, path)

def dfs_on_mst(adj_list):
    n = len(adj_list)
    visited = [False] * n
    path = []
    dfs(adj_list, 0, visited, path)
    return path

In [223]:
def metric_tsp_approximation(matrix):
    """
    An algorithm for solving the Metric TSP using minimum spanning trees and depth first search.
    
    Args:
        matrix: List[List[float]]
            An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
    
    Returns:
        path: List[int] 
            A list corresponding to the order in which to visit cities, starting from path[0] and ending 
            at path[-1] before returning to path[0].  
    """
    G = matrix_to_adj_list(matrix)
    mst = kruskal(G)
    adj_list = construct_adj_list(mst)
    for l in adj_list:
        l.sort()
    path = dfs_on_mst(adj_list)
    return path

In [224]:
grader.check("q2")

All test cases passed!
All test cases passed!


q2 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [225]:
grader.export(pdf=False, force_save=True)

<IPython.core.display.Javascript object>

/Users/jehuzepeda-silva/anaconda3/lib/python3.11/site-packages/otter/check/notebook.py:434: UserWarning: Couldn't automatically save the notebook; we recommend using File > Save & Checkpoint and then re-running this cell. The zip file returned by this call will use the last saved version of this notebook.
  warnings.warn(
